# Elektrificatie & CO₂ – Een doorlopende datastory

## Wat staat er op het spel?


Europa wil in **2030** ten minste **55 % minder CO₂‑uitstoot**. Kunnen elektrische voertuigen (EV’s) ervoor zorgen om de uitstoot in de transportsector zoveel mogelijk te verminderend. We analyseerden registratie van EV's en uitstootdata **(2010‑2023)** voor de 27 EU‑landen om deze vraag te beantwoorden:


> *Draagt het stijgende aandeel elektrische voertuigen direct bij aan lagere CO₂ uitstoot van personenauto’s?*


We verkennen twee perspectieven en gebruiken interactieve visualisaties om de data te onderbouwen.


## Elektrische voertuigen als dé oplossing?


**Perspectief 1 – EV’s als primaire oplossing.**
Dit hoofdstuk onderzoekt of een stijgend aandeel elektrische voertuigen (EV’s) leidt tot **lagere CO₂‑uitstoot** van nieuw geregistreerde auto’s.  
We gebruiken twee visualisaties:

1. **Lijngrafiek** – EV‑aandeel per land (2010‑2023)  
2. **Scatter + slider** – EV‑aandeel vs CO₂‑uitstoot per land/jaar


In [44]:
# --- Plotly → pastel kaart-renderer (robust) -------------------------
from IPython.display import HTML, display
import plotly.graph_objects as go   # go.Figure is de basisklasse

PASTEL    = "#FDEDDD"   # één uniforme pastel-kleur
CSS_CLASS = "plot-card" # class uit custom.css

def _repr_html_card(self):
    """Geeft Plotly-figuur terug als responsive kaart-HTML."""
    # 1) Pastel achtergrond in de figuur zelf
    self.update_layout(paper_bgcolor=PASTEL, plot_bgcolor=PASTEL)
    # 2) Genereer responsive HTML
    html = self.to_html(full_html=False,
                        include_plotlyjs="cdn",
                        config={"responsive": True})
    # 3) Wikkel in kaart-div
    return f'<div class="{CSS_CLASS}">{html}</div>'

def _show_card(self, *args, **kwargs):
    """Vervangt fig.show() zodat die óók het kaartje laat zien."""
    display(HTML(_repr_html_card(self)))

# ── Patchen ──────────────────────────────────────────────────────────
go.Figure._repr_html_ = _repr_html_card   # implicit display in notebooks
go.Figure.show       = _show_card         # expliciete show-calls



In [45]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display


%matplotlib inline

# --- Gecombineerde dataset inladen ---
df_full = pd.read_csv('data/ev_airquality_ready.csv')

# Kolommen hernoemen en extra velden aanmaken
df = (df_full
      .rename(columns={'EV_share': 'evShare',
                       'CO2_g_km': 'co2',
                       'TIME_PERIOD': 'year',
                       'ELC': 'elc',
                       'TOTAL': 'total'})
     )

# Compatibiliteit met eerdere variabelen
ev_share = df[['geo', 'year', 'elc', 'total']].rename(columns={'elc': 'EV', 'total': 'TOTAL'})
ev_share['EV_SHARE'] = df['evShare']

co2 = df[['geo', 'year', 'co2']].rename(columns={'co2': 'OBS_VALUE'})

START_YEAR = int(df['year'].min())
END_YEAR   = int(df['year'].max())

focus_codes = ['NL', 'DE', 'NO', 'PL', 'FR', 'ES']
focus = df[df.geo.isin(focus_codes)]

code2name = dict(NL='Nederland', DE='Duitsland', NO='Noorwegen',
                 PL='Polen', FR='Frankrijk', ES='Spanje')


In [46]:

from IPython.display import HTML

fig_line = px.line(
    focus.replace({'geo': code2name}),
    x='year', y='evShare', color='geo',
    title='EV‑aandeel in geselecteerde landen',
    labels=dict(year='Jaar', evShare='EV‑aandeel (%)', geo='Land')
)
fig_line.update_traces(mode='lines+markers')
fig_line.update_yaxes(ticksuffix='%')

# Convert to HTML and display
fig_line.show() 


Deze lijngrafiek die de evolutie van het EV-aandeel (aandeel elektrische voertuigen) toont voor verschillende Europese landen over een periode van dertien jaar, van 2010 tot en met 2023.

Wat deze visualisatie in essentie laat zien, is hoe snel of langzaam de adoptie van elektrische auto's is gegaan in verschillende landen van de Europese Unie (en Noorwegen, dat vaak wordt meegenomen vanwege zijn vooruitstrevende beleid). Het "EV-aandeel" wordt hier berekend als het percentage van elektrische auto's (ELC) ten opzichte van het totale aantal geregistreerde voertuigen (TOTAL), vermenigvuldigd met 100%.

De belangrijkste observaties zijn:

1.  **Noorwegen als koploper:** Dit land springt er direct uit met een significant en consistent hoger EV-aandeel gedurende de gehele periode. De lijn voor Noorwegen zal steil omhoog lopen en zich duidelijk afscheiden van de andere landen.

2.  **Variërende groeipatronen:** Voor de meeste andere landen zal je waarschijnlijk een geleidelijke stijging van het EV-aandeel zien, met in de latere jaren (vooral na 2018-2020) een versnelling. Echter, de helling van deze lijnen zal sterk variëren. Sommige landen, zoals Nederland of Zweden, laten mogelijk een snellere groei zien dan landen die minder actief beleid voeren.

3.  **Achterblijvers:** Landen zoals Polen en Spanje zullen lijnen vertonen die veel lager liggen en minder steil stijgen. Dit duidt op een tragere adoptie van elektrische voertuigen. De website noemt expliciet dat beleidskeuzes en de beschikbaarheid van laadinfrastructuur hier een bepalende rol spelen. In landen met minder overheidsstimulansen of een onderontwikkeld netwerk van laadpunten, is de drempel om over te stappen op elektrisch rijden hoger.

Deze visualisatie dient als een fundamenteel inzicht in de mate van elektrificatie per land voordat het verband wordt gelegd met de CO₂-uitstoot. Het laat zien dat er geen sprake is van een uniforme transitie; elk land heeft zijn eigen tempo en uitdagingen op het pad naar elektrisch rijden.

In [47]:
import plotly.graph_objects as go
import pandas as pd
from IPython.display import HTML

# ---------------------------------------------------------------------------
#  DataFrame 'focus' moet de kolommen hebben:
#  ['year', 'geo', 'evShare', 'co2']
#  geo = ISO-3166 alpha-2 (NL, BE, DE …)
# ---------------------------------------------------------------------------

# ── instellingen ────────────────────────────────────────────────────────────
FLAG_W  = 80    # FlagCDN ondersteunt w20, w40, w80, w160 …
SIZEX   = 2.0   # data-eenheden op x-as (0-25 % → 2 ≈ 8 % breedte)
SIZEY   = 6.0   # data-eenheden op y-as (50-140 g/km → 6 ≈ 7 % hoogte)

def iso_to_flag_url(iso2: str, width: int = FLAG_W) -> str:
    """Geef URL naar een PNG-vlaggetje van FlagCDN in ondersteunde breedte."""
    return f"https://flagcdn.com/w{width}/{iso2.lower()}.png"

# ISO-code → landnaam voor de hovertekst
try:
    import pycountry
    code2name = {c.alpha_2: c.name for c in pycountry.countries}
except ImportError:
    code2name = {c: c for c in focus["geo"].unique()}

# ── frames opbouwen ─────────────────────────────────────────────────────────
frames = []
for yr, grp in focus.groupby("year"):
    hover_txt = grp["geo"].map(lambda g: code2name.get(g, g))

    # onzichtbare scatter-trace (voor hover, selectie, slider)
    scatter = go.Scatter(
        x=grp["evShare"],
        y=grp["co2"],
        mode="markers",
        marker=dict(size=18, opacity=0),
        customdata=hover_txt,
        hovertemplate="<b>%{customdata}</b><br>"
                      "EV: %{x:.2f}%<br>"
                      "CO₂: %{y:.1f} g/km<extra></extra>"
    )

    # vlag-afbeeldingen op exact dezelfde x/y-posities
    images = [
        dict(
            source=iso_to_flag_url(row.geo),
            x=row.evShare, y=row.co2,
            xref="x", yref="y",
            sizex=SIZEX, sizey=SIZEY,
            xanchor="center", yanchor="middle",
            layer="above"
        )
        for _, row in grp.iterrows()
    ]

    frames.append(go.Frame(name=str(yr), data=[scatter],
                           layout=go.Layout(images=images)))

# ── basis-layout ────────────────────────────────────────────────────────────
layout = go.Layout(
    title=f"CO₂-uitstoot vs. EV-aandeel ({frames[0].name})",
    xaxis=dict(title="EV-aandeel (%)", ticksuffix="%"),
    yaxis=dict(title="Gem. CO₂ (g/km)"),
    sliders=[dict(
        steps=[dict(method="animate",
                    label=f.name,
                    args=[[f.name],
                          dict(mode="immediate",
                               frame=dict(duration=400, redraw=False),
                               transition=dict(duration=300))])
               for f in frames],
        currentvalue=dict(prefix="Jaar: ")
    )],
    images=frames[0].layout.images           # vlaggen voor het eerste frame
)

fig_scatter = go.Figure(data=frames[0].data,
                        frames=frames,
                        layout=layout)

# ── tonen / exporteren ──────────────────────────────────────────────────────
fig_scatter.show()  # toont de figuur in Jupyter Notebook


Deze visualisatie presenteert een spreidingsdiagram die de relatie tussen twee belangrijke variabelen weergeeft: het EV-aandeel en de gemiddelde CO₂-uitstoot per kilometer voor nieuw geregistreerde auto's in de betreffende landen.

De kernboodschap van deze visualisatie is het aantonen van een negatieve correlatie tussen deze twee variabelen. Dit betekent concreet:
> *Hoe hoger het EV-aandeel van een land, des te lager de gemiddelde CO₂-uitstoot per kilometer van de nieuwe auto's die daar worden geregistreerd.*

Deze visualisatie ondersteunt het idee dat elektrische voertuigen een positieve invloed hebben op de directe CO₂-uitstoot van personenauto's op de weg, en daarmee bijdragen aan het behalen van klimaatdoelstellingen. Het vormt een cruciaal bewijsstuk voor het "Elektrische voertuigen als de oplossing"-perspectief dat in het artikel wordt besproken.

### Tussenstand

Een stijgend EV‑aandeel gaat **hand in hand** met een CO₂‑daling, maar het verband is niet overal even sterk.  
Zou de **stroommix** of **voertuigefficiëntie** het effect temperen?  
Dat onderzoeken we in het volgende perspectief.


---

## Elektrificatie alléén is onvoldoende


**Perspectief 2 – Beperkte impact van EV‑adoptie.**
Dit hoofdstuk bekijkt in hoeverre de huidige adoptie van elektrische voertuigen (EV’s)
**niet altijd** resulteert in een uniforme of significante daling van de CO₂‑uitstoot
van nieuw geregistreerde auto’s.

We presenteren twee visualisaties:

1. **Lijngrafiek** – Evolutie van CO₂‑uitstoot per land (2013‑2023)  
2. **Interactieve kaart** – EV‑aandeel per land met jaarslider


In [48]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display

%matplotlib inline

# --- Gecombineerde dataset inladen ---
df_full = pd.read_csv('data/ev_airquality_ready.csv')

# Kolommen hernoemen en extra velden aanmaken
df = (df_full
      .rename(columns={'EV_share': 'evShare',
                       'CO2_g_km': 'co2',
                       'TIME_PERIOD': 'year',
                       'ELC': 'elc',
                       'TOTAL': 'total'})
     )

# Compatibiliteit met eerdere variabelen
ev_share = df[['geo', 'year', 'elc', 'total']].rename(columns={'elc': 'EV', 'total': 'TOTAL'})
ev_share['EV_SHARE'] = df['evShare']

co2 = df[['geo', 'year', 'co2']].rename(columns={'co2': 'OBS_VALUE'})

START_YEAR = int(df['year'].min())
END_YEAR   = int(df['year'].max())

focus_codes = ['NL', 'DE', 'NO', 'PL', 'FR', 'ES']
focus = df[df.geo.isin(focus_codes)]

code2name = dict(NL='Nederland', DE='Duitsland', NO='Noorwegen',
                 PL='Polen', FR='Frankrijk', ES='Spanje')


In [49]:

fig_line = px.line(focus, x='year', y='co2', color='geo',
                   title='Gem. CO₂‑uitstoot nieuwe auto’s (2013‑2023)',
                   labels=dict(year='Jaar', co2='CO₂ (g/km)', geo='Land'))
fig_line.update_traces(mode='lines+markers')
# Convert to HTML and display
fig_line.show()


De visualisatie is een lijngrafiek die de gemiddelde CO₂-uitstoot per kilometer van nieuw geregistreerde personenauto's over tijd  weergeeft, en dit voor verschillende landen. Wat je hierin ziet gebeuren is het volgende:

1. **Geen uniforme daling:** In tegenstelling tot de verwachting dat een stijgend EV-aandeel overal automatisch leidt tot een snelle daling van de uitstoot, toont deze visualisatie aan dat de daling van de CO₂-uitstoot per land sterk varieert en vaak traag of zelfs wisselvallig is. Dit suggereert dat er meer factoren meespelen dan alleen het aantal elektrische auto's.

2. **Noorwegen als uitzondering (maar met een nuance):** Hoewel Noorwegen in de eerste visualisatie de absolute koploper was in EV-adoptie, en de CO₂-uitstoot daar inderdaad snel daalt, benadrukt deze visualisatie dat dit succes gekoppeld is aan zeer sterke beleidsmaatregelen. Dit bevestigt dat de daling niet vanzelf komt.

Kortom, deze grafiek toont aan dat de weg naar lagere CO₂-uitstoot complexer is dan alleen het verhogen van het aantal elektrische auto's. Het benadrukt dat beleid, infrastructuur en de bredere marktdynamiek essentiële factoren zijn die de daadwerkelijke impact van elektrificatie bepalen.

In [50]:
iso_map = {'AT':'AUT','BE':'BEL','BG':'BGR','HR':'HRV','CY':'CYP','CZ':'CZE','DK':'DNK','EE':'EST',
           'FI':'FIN','FR':'FRA','DE':'DEU','EL':'GRC','HU':'HUN','IE':'IRL','IT':'ITA','LV':'LVA',
           'LT':'LTU','LU':'LUX','MT':'MLT','NL':'NLD','PL':'POL','PT':'PRT','RO':'ROU','SK':'SVK',
           'SI':'SVN','ES':'ESP','SE':'SWE','NO':'NOR','IS':'ISL','CH':'CHE','UK':'GBR'}
country_names = {
    'AT': 'Oostenrijk', 'BE': 'België', 'BG': 'Bulgarije', 'HR': 'Kroatië', 'CY': 'Cyprus',
    'CZ': 'Tsjechië', 'DK': 'Denemarken', 'EE': 'Estland', 'FI': 'Finland', 'FR': 'Frankrijk',
    'DE': 'Duitsland', 'EL': 'Griekenland', 'HU': 'Hongarije', 'IE': 'Ierland', 'IT': 'Italië',
    'LV': 'Letland', 'LT': 'Litouwen', 'LU': 'Luxemburg', 'MT': 'Malta', 'NL': 'Nederland',
    'PL': 'Polen', 'PT': 'Portugal', 'RO': 'Roemenië', 'SK': 'Slowakije', 'SI': 'Slovenië',
    'ES': 'Spanje', 'SE': 'Zweden', 'NO': 'Noorwegen', 'IS': 'IJsland', 'CH': 'Zwitserland',
    'UK': 'Verenigd Koninkrijk'
}
df['country_name'] = df['geo'].map(country_names)
df['iso3'] = df.geo.map(iso_map)
rows = df.dropna(subset=['iso3'])

years = sorted(rows.year.unique())
zmax = rows.evShare.quantile(0.98)

# Blauwe kleurschaal met veel meer detail voor betere zichtbaarheid van alle EV-percentages
custom_colorscale = [
    [0.0, '#f8f9fa'],      # Heel lichtgrijs voor 0%
    [0.002, '#e3f2fd'],    # Zeer lichtblauw voor 0.1%
    [0.005, '#bbdefb'],    # Lichtblauw voor 0.3%
    [0.01, '#90caf9'],     # Blauw voor 0.5%
    [0.02, '#64b5f6'],     # Middenblauw voor 1%
    [0.03, '#42a5f5'],     # Blauw voor 1.5%
    [0.05, '#2196f3'],     # Levendig blauw voor 2.5%
    [0.07, '#1e88e5'],     # Donker blauw voor 3.5%
    [0.1, '#1976d2'],      # Donkerblauw voor 5%
    [0.15, '#1565c0'],     # Zeer donkerblauw voor 7.5%
    [0.2, '#0d47a1'],      # Heel donkerblauw voor 10%
    [0.3, '#0a3d91'],      # Donker navy voor 15%
    [0.4, '#083381'],      # Navy voor 20%
    [0.5, '#062971'],      # Donker navy voor 25%
    [0.6, '#041f61'],      # Zeer donker navy voor 30%
    [0.7, '#031551'],      # Bijna zwart navy voor 35%
    [0.8, '#020b41'],      # Heel donker navy voor 40%
    [0.9, '#010531'],      # Bijna zwart voor 45%
    [1.0, '#000021']       # Zwart-navy voor 50%+
]

frames = []
for yr in years:
    sub = rows[rows.year == yr]
    frames.append(go.Frame(
        name=str(yr),
        data=[go.Choropleth(
            locations=sub.iso3,
            z=sub.evShare,
            text=sub.country_name,
            colorscale=custom_colorscale,
            zmin=0, 
            zmax=zmax,
            hovertemplate='<b>%{text}</b><br>EV-aandeel: %{z:.2f}%<br>Jaar: ' + str(yr) + '<extra></extra>',
            colorbar=dict(
                title=dict(text="EV-aandeel (%)", font=dict(size=16, color='rgb(42, 63, 95)')),
                tickfont=dict(size=14, color='rgb(42, 63, 95)'),
                len=0.85,
                thickness=25,
                x=1.01,
                tickmode='linear',
                tick0=0,
                dtick=5,
                ticksuffix='%'
            )
        )],
        layout=dict(title=dict(text=f'EV‑aandeel per Europees land ({yr})'))
    ))



# Convert to HTML and display
fig_map.show()  # toont de figuur in Jupyter Notebook

Dit is een interactieve kaart die het EV-aandeel per land toont.
Deze visualisatie versterkt het argument van het tweede perspectief ("Elektrificatie alleen is onvoldoende") door te laten zien dat de adoptie van EV's geen uniforme beweging is. Het is een gefragmenteerd proces met aanzienlijke regionale verschillen die worden beïnvloed door lokale factoren zoals beleid, economische omstandigheden en laadinfrastructuur, wat vervolgens weer invloed heeft op de CO₂-uitstoot.

---

## Resultaten & Conclusie

Binnen onze datastory stonden twee – op het eerste gezicht tegengestelde – **perspectieven** centraal:

* **Perspectief 1 – *“EV’s als primaire oplossing.”***  
  Verwacht: een duidelijk, bijna lineair verband tussen EV‑aandeel en CO₂‑daling.  
* **Perspectief 2 – *“Elektrificatie alléén is onvoldoende.”***  
  Verwacht: een zwak of inconsistent verband; andere factoren (stroommix, efficiëntie) spelen mee.

Om te beoordelen welke positie het sterkst is (of hoe ze elkaar aanvullen) tonen we drie interactieve visualisaties:

1. **EU‑trend** – laat de grote lijn zien.  
2. **Slope‑chart per land** – onthult verschillen in tempo.  
3. **ΔEV vs ΔCO₂ scatter** – test het veronderstelde causale verband direct.

Scrol, hover en filter om zelf de data te verkennen. Onder elke figuur geven we een korte interpretatie.

In [51]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display

%matplotlib inline

# --- Gecombineerde dataset inladen ---
df_full = pd.read_csv('data/ev_airquality_ready.csv')

# Kolommen hernoemen en extra velden aanmaken
df = (df_full
      .rename(columns={'EV_share': 'evShare',
                       'CO2_g_km': 'co2',
                       'TIME_PERIOD': 'year',
                       'ELC': 'elc',
                       'TOTAL': 'total'})
     )

# Compatibiliteit met eerdere variabelen
ev_share = df[['geo', 'year', 'elc', 'total']].rename(columns={'elc': 'EV', 'total': 'TOTAL'})
ev_share['EV_SHARE'] = df['evShare']

co2 = df[['geo', 'year', 'co2']].rename(columns={'co2': 'OBS_VALUE'})

START_YEAR = int(df['year'].min())
END_YEAR   = int(df['year'].max())

focus_codes = ['NL', 'DE', 'NO', 'PL', 'FR', 'ES']
focus = df[df.geo.isin(focus_codes)]

code2name = dict(NL='Nederland', DE='Duitsland', NO='Noorwegen',
                 PL='Polen', FR='Frankrijk', ES='Spanje')


In [52]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# EU-totaal: som EV en TOTAL per jaar
eu_ev = ev_share.groupby('year').agg({'EV': 'sum', 'TOTAL': 'sum'})
eu_ev['EV_SHARE'] = eu_ev['EV'] / eu_ev['TOTAL'] * 100

# EU-gemiddelde CO2 per jaar
eu_co2 = co2.groupby('year')['OBS_VALUE'].mean().rename('CO2_MEAN')

# Alles samen in één dataframe
eu = eu_ev.join(eu_co2).reset_index()

fig1 = make_subplots(specs=[[{"secondary_y": True}]])

# Lijn EV-aandeel
fig1.add_trace(
    go.Scatter(
        x=eu['year'],
        y=eu['EV_SHARE'],
        mode='lines+markers',
        name='EV-aandeel (%)',
        hovertemplate='Jaar %{x}<br>EV-aandeel %{y:.2f}%<extra></extra>'
    ),
    secondary_y=False
)

# Lijn CO2
fig1.add_trace(
    go.Scatter(
        x=eu['year'],
        y=eu['CO2_MEAN'],
        mode='lines+markers',
        line=dict(dash='dash'),
        name='CO₂ (g/km)',
        hovertemplate='Jaar %{x}<br>CO₂ %{y:.1f} g/km<extra></extra>'
    ),
    secondary_y=True
)

fig1.update_layout(
    title=f'EV-aandeel vs. CO₂ (EU-gemiddelde, {START_YEAR}–{END_YEAR})',
    xaxis_title='Jaar',
    yaxis_title='EV-aandeel (%)',
    legend_title='Indicator'
)
fig1.update_yaxes(title_text='CO₂ (g/km)', secondary_y=True)
fig_map = go.Figure(
    data=frames[0].data,
    frames=frames,
    layout=go.Layout(
        title=dict(
            text=f'EV‑aandeel per Europees land ({years[0]})',
            font=dict(size=22, color='rgb(42, 63, 95)', family='Arial, sans-serif'),
            x=0.5,
            xanchor='center',
            y=0.95,
            yanchor='top'
        ),
        geo=dict(
            scope='europe', 
            projection_type='natural earth',
            showframe=False, 
            showcountries=True,
            countrycolor='#7fb3d3',
            countrywidth=0.8,
            bgcolor='rgba(0,0,0,0)',
            landcolor='#f0f8ff',
            oceancolor='#cce7ff',
            showocean=True,
            showlakes=True,
            lakecolor='#cce7ff'
        ),
        paper_bgcolor='#f8fbff',
        plot_bgcolor='#f8fbff',
        font=dict(family='Arial, sans-serif', color='#2c3e50'),
        sliders=[dict(
            steps=[dict(
                method='animate', 
                label=str(y),
                args=[[str(y)], dict(
                    mode='immediate',
                    frame=dict(duration=300, redraw=True),
                    transition=dict(duration=200, easing='cubic-in-out')
                )]
            ) for y in years],
            currentvalue=dict(
                prefix='Jaar: ',
                font=dict(size=18, color='rgb(42, 63, 95)', weight='bold')
            ),
            pad=dict(t=60, b=20, l=40, r=40),
            len=0.85,
            x=0.075,
            xanchor='left',
            bgcolor='rgba(255,255,255,0.95)',
            bordercolor='#2196f3',
            borderwidth=2,
            font=dict(color='rgb(42, 63, 95)', size=14)
        )],
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            x=0.075,
            y=0.02,
            xanchor='left',
            yanchor='bottom',
            buttons=[
                dict(
                    label='▶',
                    method='animate',
                    args=[None, dict(
                        frame=dict(duration=1000, redraw=True),
                        transition=dict(duration=400),
                        fromcurrent=True,
                        mode='immediate'
                    )]
                ),
                dict(
                    label='⏸',
                    method='animate',
                    args=[[None], dict(
                        frame=dict(duration=0, redraw=False),
                        mode='immediate',
                        transition=dict(duration=0)
                    )]
                )
            ],
            bgcolor='rgba(255,255,255,0.95)',
            bordercolor='rgb(42, 63, 95)',
            borderwidth=2,
            font=dict(color='rgb(42, 63, 95)', size=13, weight='bold')
        )],
        margin=dict(l=40, r=40, t=80, b=80)
    )
)

fig1.show()  # toont de figuur in Jupyter Notebook


De figuur toont de ontwikkeling van het aandeel elektrische voertuigen (in blauw) en de gemiddelde CO₂-uitstoot per nieuw voertuig op basis van typegoedkeuring (gestippelde lijn) binnen de EU. Sinds 2014 vertoont het EV-aandeel een duidelijke exponentiële groei. Tegelijkertijd daalt de gemiddelde CO₂-uitstoot geleidelijk over dezelfde periode.

**Analyse:**
De richting van beide trends is consistent met het narratief dat elektrificatie bijdraagt aan emissiereductie, een argument dat aansluit bij Perspectief 1, waarin de elektrische auto wordt gepresenteerd als motor van verandering. Echter, een nadere blik op de helling van de curves wijst op een belangrijk spanningsveld: de groei van het EV-aandeel versnelt duidelijk sneller dan de daling in CO₂-uitstoot.

**Interpretatie en relevantie:**
Dit verschil suggereert dat de toename van elektrische voertuigen slechts gedeeltelijk doorwerkt in lagere gemiddelde uitstoot. Dit impliceert dat andere factoren – zoals de uitstoot van niet-elektrische voertuigen, de precieze methodologie van typegoedkeuringsmetingen, of de emissie-intensiteit van de gebruikte energie – een modererende rol spelen. Daarmee biedt de figuur tevens ondersteuning aan Perspectief 2, waarin elektrificatie wordt beschouwd als noodzakelijke, maar onvoldoende voorwaarde voor substantiële emissiereductie op systeemniveau.

In [53]:
# ---------- Figuur 2 (interactief, responsive kaartje) ----------
import plotly.graph_objects as go

# 1. Selecteer eerste en laatste jaar in de dataset
min_year, max_year = int(df["year"].min()), int(df["year"].max())

# 2. Maak tweejaren-subset en zorg dat elk land in beide jaren zit
df_slope = (
    df[df["year"].isin([min_year, max_year])]
      .loc[:, ["geo", "year", "evShare"]]
      .rename(columns={"evShare": "EV_SHARE"})
)
valid_geo = (
    df_slope.groupby("geo")["year"]
            .nunique()
            .pipe(lambda s: s[s == 2].index)
)
df_slope = df_slope[df_slope["geo"].isin(valid_geo)]

# 3. Sorteer zodat hoogste EV-aandelen bovenaan staan
df_slope = df_slope.sort_values(["year", "EV_SHARE"])

# 4. Plot lijnen
fig2 = go.Figure()
for country, grp in df_slope.groupby("geo"):
    y_vals = grp.sort_values("year")["EV_SHARE"].values
    fig2.add_trace(
        go.Scatter(
            x=[str(min_year), str(max_year)],
            y=y_vals,
            mode="lines+markers",
            name=country,
            hovertemplate=f"<b>{country}</b><br>%{{x}}: %{{y:.2f}}%<extra></extra>",
            line=dict(width=2)
        )
    )

# 5. Dropdown
buttons = [{
    "label": "Alle landen",
    "method": "update",
    "args": [{"visible": [True] * len(fig2.data)},
             {"title": f"EV-adoptie per land: {min_year} → {max_year}"}]
}]
for i, trace in enumerate(fig2.data):
    vis = [False] * len(fig2.data)
    vis[i] = True
    buttons.append({
        "label": trace.name,
        "method": "update",
        "args": [{"visible": vis},
                 {"title": f"EV-adoptie: {trace.name} ({min_year} → {max_year})"}]
    })

# 6. Lay-out – géén vaste width/height, wel autosize
fig2.update_layout(
    title=f"EV-adoptie per land: {min_year} → {max_year}",
    xaxis_title="Jaar",
    yaxis_title="EV-aandeel (%)",
    xaxis=dict(type="category"),
    template="plotly",
    hovermode="closest",
    autosize=True,
    margin=dict(l=60, r=40, t=70, b=60),
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "showactive": True,
        "x": 1.05,
        "y": 1.15
    }]
)

fig2.show()          # patch toont ’m als responsive kaart


Via de dropdown of legenda kan een specifiek land worden geselecteerd om individuele trends in elektrificatie te analyseren. (EU27_2020 is het gemiddelde van 27 landen)

Belangrijkste bevindingen:

1. De Scandinavische landen – Denemarken, Zweden en met name Noorwegen – laten een duidelijke en consistente toename zien in het aandeel elektrische voertuigen.
2. In tegenstelling daarmee blijven landen in Zuid- en Oost-Europa, zoals Bulgarije, Roemenië en Griekenland, gedurende de onderzochte periode vrijwel stabiel, met slechts minimale vooruitgang.
3. Het EU-gemiddelde suggereert een geleidelijke stijging, maar maskeert daarmee de aanzienlijke variatie tussen lidstaten.

Interpretatie en relevantie:
De uiteenlopende nationale trends wijzen op structurele verschillen in overheidsbeleid, stimuleringsmaatregelen, economische draagkracht en laadnetwerkinfrastructuur. Deze spreiding vormt een belangrijk aanknopingspunt voor Perspectief 2

In [54]:
# ---------- Figuur 3 (interactief, responsive kaartje) ----------
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# 1. Kies eerste en laatste jaar
min_year, max_year = START_YEAR, END_YEAR

# 2. Tweejaren-subset
chg = df[df["year"].isin([min_year, max_year])].copy()

# 3. Wide-form
pivot_ev  = chg.pivot(index="geo", columns="year", values="evShare")
pivot_co2 = chg.pivot(index="geo", columns="year", values="co2")

# 4. Veranderingen
delta_ev  = pivot_ev[max_year]  - pivot_ev[min_year]
delta_co2 = pivot_co2[min_year] - pivot_co2[max_year]

scatter_df = (
    pd.DataFrame({
        "Δ_EV_share_pp": delta_ev,
        "Δ_CO2_gkm"   : delta_co2
    })
    .dropna()
    .reset_index()
    .rename(columns={"geo": "Land"})
)

# 5. Trendline
m, b = np.polyfit(scatter_df["Δ_EV_share_pp"], scatter_df["Δ_CO2_gkm"], 1)
x_line = np.linspace(scatter_df["Δ_EV_share_pp"].min(),
                     scatter_df["Δ_EV_share_pp"].max(), 100)
r = scatter_df[["Δ_EV_share_pp", "Δ_CO2_gkm"]].corr().iloc[0, 1]

# 6. Plot
fig3 = px.scatter(
    scatter_df,
    x="Δ_EV_share_pp",
    y="Δ_CO2_gkm",
    text="Land",
    labels={
        "Δ_EV_share_pp": "Toename EV-aandeel (procentpunten)",
        "Δ_CO2_gkm": "Daling CO₂ (g/km)"
    },
    template="plotly"
)

fig3.add_trace(
    go.Scatter(
        x=x_line,
        y=m * x_line + b,
        mode="lines",
        line=dict(dash="dash", width=1.5),
        name="Trendline",
        hoverinfo="skip"
    )
)

fig3.update_traces(
    marker=dict(size=10, line=dict(width=0.5, color="DarkSlateGrey"))
)
fig3.update_layout(
    title=f"Grotere EV-groei ⇒ grotere CO₂-daling?  (Pearson r = {r:.2f})",
    showlegend=False,
    autosize=True,
    margin=dict(l=60, r=40, t=70, b=60)
)

fig3.show()          # responsive kaart


In deze scatterplot vertegenwoordigt elk punt een afzonderlijk land. De gestippelde regressielijn geeft de best passende lineaire relatie weer tussen de toename van het aandeel elektrische voertuigen (ΔEV) en de daling in gemiddelde CO₂-uitstoot van nieuw geregistreerde auto's (ΔCO₂).

De Pearson-correlatiecoëfficiënt bedraagt circa 0,76, wat wijst op een sterke en positieve samenhang tussen elektrificatie en emissiereductie op nationaal niveau.

**Belangrijke nuances:**
Hoewel de algemene trend duidelijk positief is, vertonen sommige landen aanzienlijke afwijkingen van de regressielijn. Zo zijn er landen die een sterke groei in elektrische voertuigen realiseren, maar waarbij de CO₂-uitstoot slechts beperkt daalt. Dit duidt mogelijk op een koolstofintensieve elektriciteitsmix of een relatief inefficiënt resterend wagenpark. Omgekeerd zijn er ook landen met beperkte EV-groei die toch een substantiële reductie in CO₂-emissies bereiken, mogelijk door verbeteringen in brandstofefficiëntie of snelle vervanging van oudere voertuigen.

**Interpretatie en relevantie:**
De grafiek onderstreept dat elektrificatie een belangrijke factor is in de reductie van CO₂-uitstoot, maar geen allesomvattende verklaring biedt. De uiteindelijke klimaatimpact van elektrische mobiliteit wordt mede bepaald door de koolstofintensiteit van de stroomvoorziening, de technologische efficiëntie van nieuw verkochte voertuigen en de vernieuwing van het bestaande wagenpark.

## Wie heeft er gelijk?

| | Observatie uit data | Steun voor perspectief |
|---|---|---|
| **EV ↘ CO₂ op EU‑niveau** | Ja, maar matige hellingshoek | 1 **en** 2 |
| **Grote land‑tot‑land variatie** | Ja, factor > 10 in groei | **2** |
| **Strak lineair verband ΔEV–ΔCO₂** | Ja, maar niet perfect lineair, r ≈ 0.76 | **2** (nuancerend) |

### Conclusie  
*Elektrificatie is een krachtige hefboom (Perspectief 1), maar haar effectiviteit wordt gemoduleerd door stroommix, beleid en voertuig­efficiëntie (Perspectief 2). In de huidige data lijkt een “50‑50‑verklaring” het meest realistisch: EV‑groei levert een noodzakelijke, maar niet voldoende, bijdrage aan de dalende CO₂‑trend.*

### Aanbeveling  
Versnel simultaan:  
1. **EV‑adoptie** (aanschafsubsidies, laadinfrastructuur), én  
2. **Verduurzaming van elektriciteit** + **strengere efficiëntienormen**.  

Alleen dan convergeert de blauwe EV‑lijn in Figuur 1 naar een even steile daling in de gestippelde CO₂‑lijn.

### Samenvatting
De elektrische auto wordt vaak gepresenteerd als dé oplossing tegen klimaatverandering: stil, schoon en cruciaal voor het behalen van de Europese klimaatdoelen. Maar is dat beeld wel terecht?

Onze analyse van data uit 27 Europese landen (2010–2023) laat een gemengd beeld zien. In landen als Noorwegen – koploper in elektrificatie – daalt de CO₂-uitstoot van nieuwe auto's sterk. Hier lijkt de elektrische auto daadwerkelijk het verschil te maken.

Toch is het beeld niet overal zo rooskleurig. In Nederland, Duitsland en Frankrijk groeit het aantal elektrische auto's, maar de uitstoot daalt langzamer en soms nauwelijks. Dat roept de vraag op: is elektrificatie genoeg, of zijn bredere veranderingen nodig – zoals groene stroom en duurzamere productie?

De cijfers vertellen twee verhalen. Aan ons de keuze: zien we de elektrische auto als dé held, of als slechts één onderdeel van een grotere klimaatoplossing?